In [ ]:

import IPython
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np 
import numpy.random as npr
import scipy as sp
import scipy.linalg as sl
from scipy.fft import fft
from scipy.fft import ifft
plt.rcParams["animation.html"]="jshtml"



La fonction $\textit{Kin(Nx)}$ renvoie le vecteur $[0,1,4,9..., (Nx/2)^2,- (Nx/2)^2, ...-1]$. Il pourra être utilisé lors du calcul de la dérivée seconde par TF. Elle pourra être remplacée par fftfreq par moment 

In [ ]:
def Kin (Nx):
    
    Nx_2 = int((Nx/2)*(Nx%2==0) + ((Nx-1)/2)*(Nx%2==1)) +1
    K = np.zeros((Nx), dtype='complex')
    for i in range(0,Nx_2-1):
        K[i] = 0.5*(2*np.pi/L)**2 *i*i
        print(i)
    for i in range(Nx_2,Nx-1):
        K[i] = 0.5*(2*np.pi/L)**2 *(i-Nx)*(i-Nx)
        print(i)
    return K

La fonction barriere_reg renvoie un potentiel barrière avec une régularisation de sorte à ce que le potentiel soit dans $C^\infty_C$. 
L'idée est de rajouter à un potentiel barrière classique entre $s$ et $s+l$ des splins $C^\infty_C$. allant à $0$ en $\varepsilon$.
Cela régularise la solution et évite des problèmes éventuelles aussi bien théorique que numérique lorsque l'on utilise un potentiel discontinu.

In [ ]:
splin_exp = lambda x: np.exp(-1/x)*(x>0) / (np.exp(-1/x)*(x>0)+ np.exp(-1/(1-x))*(x>0))
raccord = lambda x,p1,p2,a,b: (1-splin_exp((x-a)/(b-a)))*p1 + splin_exp((x-a)/(b-a))*p2 

def barriere_reg(X,t,x1,x2,epsi):
    V=np.zeros(X.size)

    for i in range(X.size) :
        x = X[i]

        if(x<x1-epsi):                      V[i] = 0 
        elif((x1-epsi<x)  and (x<x1+epsi)): V[i] = raccord(x,0,V0,x1-epsi,x1+epsi)
        elif((x1+epsi<x)  and (x<x2-epsi)): V[i] = V0
        elif((x2-epsi<x)  and (x<x2+epsi)): V[i] = raccord(x,V0,0,x2-epsi,x2+epsi)
        elif(x2+epsi<x):                    V[i] = 0

    return V

la fonction $\textit{Concentration de masse}$ va nous donner la norme de la fonction psi compris entre $a$ et $b$ en fonction des paramètres numérique $L$ et $Nx$ du problème.

In [ ]:
def Concentration_de_masse(psi,Nx,L,a,b):
    l = np.abs(b-a);    occupation  = l/(2*L) 
    an = int((np.abs(a+L)/(2*L))*Nx) ; bn = int((np.abs(b+L)/(2*L))*Nx)
    N = np.abs(bn-an)
    print(l,N,an,bn)
    psi_l = psi[an:bn]
    masse = np.sqrt(l/N)*np.linalg.norm(psi_l)
    return masse

La fonction $\textit{Jauges}$ va servir d'analyse de l'ensemble $(\psi_t)_t$ au travers de divers fonctions

In [ ]:

def Jauges(psi,Nx,L,t):
    dx = 1/Nx
    I = np.linspace(-L,L,Nx); Kinetic = (0.5*(2*np.pi/L)**2) *np.fft.fftfreq(Nx, dx)*np.fft.fftfreq(Nx, dx)
    print("norme de psi sur [-L,L] = ", np.sqrt(2*L/Nx)*np.linalg.norm(psi))
    print("norme de psi en dehors du puit = ", Concentration_de_masse(psi=psi,Nx=Nx,L=L,a=s,b=L)+Concentration_de_masse(psi=psi,Nx=Nx,L=L,a=-s,b=-L) )
    print("Energie de psi = ",np.sqrt(2*L/Nx)*np.linalg.norm(np.abs( ifft( Kinetic * fft(V(I,t) *psi)))) )
    print("============================== ",t," ===========================")


la fonction $\textit{dynamics-fft-diss}$ calcul la solution de l'équation de schrödinger par méthode semi-spectral en espace et splitting en temps. 
la fonction sauvegarde en chaque pas de temps la solution $\psi_t$ et sa transformée de fourier $\phi_t = \mathcal{F}(\psi_t)$

In [ ]:
def dynamics_fft_diss(psi0_fun=(lambda x: np.exp(-x**2)), V_fun=(lambda x,t: 0), L=10, Nx=100, T=4, Nt=100):

    dt = T/Nt; dx = L/Nx
    # Kinetic = Kin(Nx)
    Kinetic = (0.5*(2*np.pi/L)**2) *np.fft.fftfreq(Nx, dx)*np.fft.fftfreq(Nx, dx)
    I = np.linspace(-L, L,Nx,endpoint=False)
    Psi_T = np.zeros((Nx,Nt), dtype="complex")
    Phi_T = np.zeros((Nx,Nt), dtype="complex")

    Psi_T[:,0]=psi0_fun(I)
    
    
    for i in range(1,Nt):
        ti = dt*i
        Phi_T[:,i] = (np.exp(-1j*Kinetic*dt)) * fft(np.exp(-1j*V_fun(I,ti)*dt) * Psi_T[:,i-1])
        Psi_T[:,i] = ifft(Phi_T[:,i])
        # print(ti,np.sqrt(L/Nx)*np.linalg.norm(Psi_T[:,i]))
    return Psi_T, Phi_T

La fonction $\textit{plot-psi}$ va crée une animation de notre simulation obtenu à partir de $\textit{dynamics-fft-diss}$

In [ ]:

def plot_psi(psi, duration=10, frames_per_second=30, L=10, show_potential=False):
    
    fig, ax = plt.subplots()
    t_data = np.linspace(0, 1, np.size(psi, 1)) # 1 is arbitrary here
    x_data = np.linspace(-L,L,  np.size(psi,0))
    # set the min and maximum values of the plot, to scale the axis
    m = min(0, np.min(np.real(psi)), np.min(np.imag(psi)))
    M = np.max(np.abs(psi))
    
    # set the axis once and for all
    ax.set(xlim=[-L,L], ylim=[m,M], xlabel='x', ylabel='psi')
    
    # dummy plots, to update during the animation
    real_plot = ax.plot(x_data, np.real(psi[:, 0]), label='Real')[0]
    imag_plot = ax.plot(x_data, np.imag(psi[:, 0]), label='Imag')[0]
    abs_plot  = ax.plot(x_data, np.abs(psi[:, 0]), label='Abs')[0]
    if(show_potential):V_plot  =   ax.plot(x_data, V(x_data,0), label='V')[0]
    ax.legend()

    # define update function as an internal function (that can access the variables defined before)
    # will be called with frame=0...(duration*frames_per_second)-1
    def update(frame):
        IPython.display.clear_output(wait=True)
        print(frame)
        # get the data by linear interpolation
        t = frame / (duration * frames_per_second)
        psi_t = np.array([np.interp(t, t_data, psi[i, :]) for i in range(np.size(psi,0))])
        # update the plots
        real_plot.set_ydata(np.real(psi_t))
        imag_plot.set_ydata(np.imag(psi_t))
        abs_plot.set_ydata(np.abs(psi_t))

    ani = animation.FuncAnimation(fig=fig, func=update, frames=duration*frames_per_second, interval=1000/frames_per_second)
    return ani


$\large{Développement-de-la-barrière-de-potentiel}$: $\\ $
Avec la fonction $\textit{Barriere-reg}$, nous allons pouvoir comparer le bruit causé par un choc entre une onde incidente et un potentiel discontinu avec un choc entre une incidente et un potentiel $C^\infty$.
Avec la fonction $\textit{Concentration-de-masse}$, nous allons pouvoir étudier la masse qui est transmise par effet tunnel au travers de la barrière. $\\$ 

On note que cette étude peut s'étendre au cas du puit de potentiel lorsque l'on pose deux barrière des deux côtes de l'onde.

In [ ]:
a=1; kx=1000; x0=0
psi0= lambda x: 1/(np.sqrt(2*np.pi *a*a)) *np.exp(-((x-x0)*(x-x0))/(2*a*a)) *np.exp(1j*kx*(x-x0))

L=20; Nx=2000; Nt=5000; T=40
l=0.2; s=4; V0=10; epsi=0.01

# V = lambda x,t : V0*(x<=s+l )*(x>=s)                    #barriere discontinue
V = lambda x,t : barriere_reg(x,t,s,s+l,epsi)  #barriere C infty


psi,phi = dynamics_fft_diss(psi0_fun=psi0,V_fun=V, L=L, Nx=Nx, T=T, Nt=Nt)
J =np.linspace(-L,L,Nx, endpoint=False)
dt = 1/Nt
for k in range(0,Nt,10):
    Jauges(psi[:,k],Nx,L,dt*k)

In [ ]:
show_sol = True
if(show_sol):
    anime = plot_psi(psi,L=L, duration=10, frames_per_second=20,show_potential=True)
else:
    anime = plot_psi(phi,L=L, duration=10, frames_per_second=20)

plt.close()
anime

$\large{Développement-du-puit-de-potentiel}$: $\\ $
On regardera deux puits de potentiels: $\\$
1) $V$ est constant en dehors d'un voisinage de l'origine. $\\$
2) $V$ est composé de deux barrière de potentiel relativement épaisse

$\\$

$\Large{Bug-reports} \\$
Si $V0$ petit et $k$ grand, on lieu d'avoir un effet tunnel, l'onde prend un effet de reflection-$\large{Transmission}$ 

In [ ]:
a=1; kx=1000; x0=0
psi0= lambda x: 1/(np.sqrt(2*np.pi *a*a)) *np.exp(-((x-x0)*(x-x0))/(2*a*a)) *np.exp(1j*kx*(x-x0))

L=20; Nx=2000; Nt=20000; T=400
l=3; s=L/2; V0=100; epsi=0.01

V = lambda x,t : V0*(x>L/2) + V0*(x<-L/2)                                       # potentiel constant en dehors de [-L/2, L/2]
V = lambda x,t : barriere_reg(x,t,s,s+l,epsi) + barriere_reg(x,t,-s-l,-s,epsi)  # barriere de potentiel (régulière)


psi,phi = dynamics_fft_diss(psi0_fun=psi0,V_fun=V, L=L, Nx=Nx, T=T, Nt=Nt)
J =np.linspace(-L,L,Nx, endpoint=False)
dt = 1/Nt
for k in range(0,Nt,10):
    Jauges(psi[:,k],Nx,L,dt*k)

In [ ]:
show_sol = True
if(show_sol):
    anime = plot_psi(psi,L=L, duration=10, frames_per_second=20,show_potential=True)
else:
    anime = plot_psi(phi,L=L, duration=10, frames_per_second=20)

plt.close()
anime

$\Large{Développement-2D} \\$
-toolkit(Norme, Kin_2D, dyn_2D,plot_psi_2D) $\\$
-problème de mémoire -> red_x,red_y,savefile et psi_temp $\\$
-résultats intéressants: barrière/puit, CI anneau... $\\$
-bug : dispersion bizarre (maillage ? code ? )

In [ ]:

from scipy.fft import fft2
from scipy.fft import ifft2
import time

from mpl_toolkits.mplot3d import Axes3D


### Necessaire si l'on prend Nx ou Nt grand
savefile = 10
red_x = 4 ;red_y=4

def Norme (Nx,Ny,Lx,Ly,psi):
    return np.sqrt(Lx/Nx)*np.sqrt(Ly/Ny)*np.linalg.norm(psi)

def Projection_red (Nx,Ny,psi,red_x=red_x,red_y=red_y):
    sol_red = np.zeros((int(Nx/red_x),int(Ny/red_y)), dtype="complex")
    for i in range(0,Nx,red_x):
        for j in range(0,Ny,red_y):
            sol_red[int(i/red_x),int(j/red_y)] = psi[i,j]
    return sol_red


def Kin_2D(Nx,Ny):
    Nx_2 = int((Nx/2)*(Nx%2==0) + ((Nx-1)/2)*(Nx%2==1)) 
    Ny_2 = int((Ny/2)*(Ny%2==0) + ((Ny-1)/2)*(Ny%2==1))

    K = np.zeros((Nx,Ny), dtype='complex')
    for i in range(0,Nx_2):
        for j in range(0,Ny_2):
            K[i,j] = 0.5*(2*np.pi/L)**2 *i*j
            
        for j in range(Ny_2,Ny-1):
            K[i,j] = 0.5*(2*np.pi/L)**2 *i*(j-Ny)
    
    for i in range(Nx_2,Nx-1):
        for j in range(0,Ny_2):
            K[i,j] = 0.5*(2*np.pi/L)**2 *(i-Nx)*j
            
        for j in range(Ny_2,Ny-1):
            K[i,j] = 0.5*(2*np.pi/L)**2 *(i-Nx)*(j-Ny)

    return K


def dynamics_2D(psi0_fun=(lambda x,y: np.exp(-(x*x+y*y)**2)), V_fun=(lambda x,y,t: 0), Lx=10, Ly=10, Nx=100, Ny=100, T=4, Nt=100):


    Kinetic = Kin_2D(Nx,Ny)
    I = np.linspace(-Lx,Lx,Nx); J = np.linspace(-Ly,Ly,Ny).reshape(-1,1)
    Psi_T = np.zeros((int(Nx/red_x),int(Ny/red_y),Nt), dtype="complex")
    Psi_temp =np.zeros((Nx,Ny,2), dtype="complex"); Phi_temp = np.zeros((Nx,Ny), dtype="complex")
    Phi_T = np.zeros((int(Nx/red_x),int(Ny/red_y),Nt), dtype="complex")
    dt = T/Nt

    Psi_temp[:,:,0]=psi0_fun(I,J); Psi_T[:,:,0]= Projection_red(Nx,Ny,Psi_temp[:,:,0])
    norm = np.sqrt(Lx/Nx)*np.sqrt(Ly/Ny)*np.linalg.norm(Psi_temp[:,:,0])
    
    diff_Norm = np.zeros(Nt); Energie = np.zeros(Nt)

    for i in range(1,savefile*Nt):
        ti = dt*i
        Phi_temp[:,:]= np.exp(-1j*Kinetic*dt)* fft2(np.exp(-1j*V_fun(I,J,ti)*ti) *Psi_temp[:,:,0])
        Psi_temp[:,:,1]= ifft2(Phi_temp[:,:])
        if(i%savefile==0) : 
            Psi_T[:,:,int(i/savefile)]  = Projection_red(Nx,Ny,Psi_temp[:,:,1]); 
            Phi_T[:,:,int(i/savefile)]  = Projection_red(Nx,Ny,Phi_temp[:,:]); 

            diff_Norm[int(i/savefile)]  = np.log(np.abs(norm -Norme(Nx,Ny,Lx,Ly, Psi_temp[:,:,1])))
            Energie[int(i/savefile)]    = np.sqrt(Lx/Nx)*np.sqrt(Ly/Ny)*np.linalg.norm(ifft2( Kinetic * fft2(V_fun(I,J,ti) *Psi_temp[:,:,1])))

            print(i,diff_Norm[int(i/savefile)], Energie[int(i/savefile)])
        Psi_temp[:,:,0] = Psi_temp[:,:,1]

    return Psi_T,Phi_T, diff_Norm, Energie



In [80]:


def plot_psi_1D(psi, duration=10, frames_per_second=30, L=10):
    
    fig, ax = plt.subplots()
    t_data = np.linspace(0, 1, np.size(psi, 1)) # 1 is arbitrary here
    x_data = np.linspace(-L,L,np.size(psi,0), endpoint=False)
    # set the min and maximum values of the plot, to scale the axis
    m = min(0, np.min(np.real(psi)), np.min(np.imag(psi)))
    M = np.max(np.abs(psi))
    
    # set the axis once and for all
    ax.set(xlim=[-L,L], ylim=[m,M], xlabel='x', ylabel='psi')
    
    # dummy plots, to update during the animation
    real_plot = ax.plot(x_data, np.real(psi[:, 0]), label='Real')[0]
    imag_plot = ax.plot(x_data, np.imag(psi[:, 0]), label='Imag')[0]
    abs_plot  = ax.plot(x_data, np.abs(psi[:, 0]), label='Abs')[0]
    ax.legend()

    # define update function as an internal function (that can access the variables defined before)
    # will be called with frame=0...(duration*frames_per_second)-1
    def update(frame):
        print(frame)
        # get the data by linear interpolation
        t = frame / (duration * frames_per_second)
        psi_t = np.array([np.interp(t, t_data, psi[i, :]) for i in range(np.size(psi,0))])
        # update the plots
        real_plot.set_ydata(np.real(psi_t))
        imag_plot.set_ydata(np.imag(psi_t))
        abs_plot.set_ydata(np.abs(psi_t))

    ani = animation.FuncAnimation(fig=fig, func=update, frames=duration*frames_per_second, interval=1000/frames_per_second)
    return ani


def plot_psi_2D(psi, duration=10, frames_per_second=30, Lx=10, Ly=10):
    
    fig, ax = plt.subplots()
    

    ims=[]
    for i in range(60):
        im = ax.imshow(np.abs(psi[:,:,i]),extent=[-Lx, Lx, -Ly, Ly], animated=True)
        if i == 0:
            ax.imshow(np.abs(psi[:,:,i]),extent=[-Lx, Lx, -Ly, Ly] )
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, blit=True,repeat_delay=1000)
    return ani

In [81]:
##paramètres du problèmes
r=0.5;  a=r*np.sqrt(2*np.log(2));   kx=0; ky=0; x0=0; y0=0
N=1000;  L=20;   
Lx=L;   Nx=N;       Ly=L;   Ny=N

L = max(Lx,Ly)
Nt=5000;     T=100
l=np.sqrt(100);      V0=1

## quelques condition initiales intéressantes

# psi0 = lambda x,y: np.exp(-a*(x*x + y*y)) *np.exp(1j*kx*(x-x0)) *np.exp(1j*ky*(y-y0)) #gaussienne centrée en (x0,y0) de variance a
# psi0 = lambda x,y:  np.exp(1j*kx*(x-x0))                                              #onde classique 
psi0 = lambda x,y : 2/(np.sqrt(2*np.pi*a*a)- np.sqrt(np.pi*a*a))* np.exp(-(x*x + y*y)/(2*a*a))*(1-np.exp(-(x*x + y*y)/(2*a*a)))*np.exp(1j*kx*(x-x0)) *np.exp(1j*ky*(y-y0))  #cercle autour de l'origine
# psi0= lambda x: np.exp(-a*(x-x0)*(x-x0)) *np.exp(1j*k*(x-x0))                         #gaussienne en x indep de y


## quelques potentiel intéressants
V = lambda x,y,t : V0*((x*x + y*y)>(l)**2) -V0*((x*x + y*y)<=(l)**2)*((x*x + y*y)>=-(l)**2)     # puit d energie
# V = lambda x,y,t : V0*((x*x + y*y)<=(L/2+l)**2)*((x*x+y*y)>=(L/2)**2)                         # barriere (effet tunnel)
# V = lambda x,y,t : -5*V0/np.abs(x*x + y*y)                                                    # potentiel de coulomb
# V = lambda x,y,t : 1*np.cos(2*np.pi*x/L)                                                      # potentiel periodique
# V = lambda x,y,t : 0 * x * y                                                                  # potentiel nul


In [82]:

I = np.linspace(-Lx,Lx,Nx); J = np.linspace(-Ly,Ly,Ny).reshape(-1,1); Time = np.arange(0,Nt)
psi,phi, diff_norm, Ener = dynamics_2D(psi0_fun=psi0,V_fun=V, Lx=Lx,Ly=Ly, Nx=Nx, Ny=Ny, T=T, Nt=Nt)


## tranche de la simulation en (x,0) ou (0,y)
# anime_1D_y = plot_psi_1D(psi[:,Ny_2,:], L=Lx,  duration=2*T, frames_per_second=60)
# anime_1D_x = plot_psi_1D(psi[Nx_2,:],   L=Ly,  duration=2*T, frames_per_second=60)

#### Si problème de mémoire -> red_x,red_y et/ou savefile


KeyboardInterrupt: 

In [ ]:
show_sol = True
if(show_sol):
    anime = plot_psi_2D(psi,Lx=Lx,   Ly=Ly, duration=2*T, frames_per_second=60)
else:  
    anime = plot_psi_2D(phi,Lx=Lx,   Ly=Ly, duration=2*T, frames_per_second=60)

plt.close()
anime